# Pix2Pix on Facade Dataset — Demo Notebook

This notebook demonstrates how to train and evaluate a Pix2Pix model for image-to-image translation on the **CMP Facade Dataset**. The model learns to generate building facade labels from photos using conditional GANs.

We use a standard U-Net-based generator and PatchGAN-based discriminator. Training is done using the combined L1 + adversarial loss, following the original Pix2Pix paper.

---


In [ ]:
# TensorFlow and helper imports
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import from your module files
from model import Pix2Pix_Model
from dataset_preprocess import train_dataset, test_dataset
from utils import train_model, compute_metrics, evaluate_on_testset


## Reproducibility

We set random seeds for consistent results across runs.


In [ ]:
seed = 32
keras.backend.clear_session()
tf.random.set_seed(seed)
np.random.seed(seed)


## Configuration

Define number of epochs, learning rate, and optimizers for generator and discriminator.


In [ ]:
NUM_EPOCHS = 200
LEARNING_RATE = 2e-4
BETA_1 = 0.5

model_pix2pix = Pix2Pix_Model()
gen_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)
disc_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)


## Train Pix2Pix Model

Start training the generator and discriminator on the facades dataset.
Losses and gradient norms will be logged every 10 epochs.


In [ ]:
gen_losses, disc_losses, gen_grad_norms, disc_grad_norms, gen_gan_losses, gen_l1_losses = \
    train_model(model_pix2pix, gen_optimizer, disc_optimizer,
                train_dataset, NUM_EPOCHS, clip_norm=False, max_norm=1.0)


## Training Metrics

Let's plot:
- Generator and Discriminator Loss
- Generator GAN loss and L1 loss
- Gradient norms for both networks


In [ ]:
df = pd.DataFrame({
    "gen_loss": gen_losses,
    "disc_loss": disc_losses,
    "gen_gan": gen_gan_losses,
    "gen_l1": gen_l1_losses,
    "gen_grad_norm": gen_grad_norms,
    "disc_grad_norm": disc_grad_norms
})

# Plot gradient norms
df[['gen_grad_norm', 'disc_grad_norm']].plot(figsize=(8, 5), title="Gradient Norms")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("L2 Norm")
plt.show()

# Plot losses
df[['gen_loss', 'disc_loss', 'gen_gan', 'gen_l1']].plot(figsize=(8, 5), title="Generator and Discriminator Loss")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


## Evaluate Pix2Pix on Test Set

We compute:
- **PSNR (Peak Signal-to-Noise Ratio)**
- **SSIM (Structural Similarity Index)**

Then visualize predictions from a few test samples.


In [ ]:
compute_metrics(model_pix2pix, test_dataset)
evaluate_on_testset(model_pix2pix, test_dataset, num_examples=10, save_images=False)


## Conclusion

In this notebook, we:

- Trained a **Pix2Pix** model on the **CMP Facade Dataset**
- Used a U-Net generator and PatchGAN discriminator
- Monitored training loss, gradient norms, and visual outputs
- Evaluated the model using **PSNR** and **SSIM**

This project demonstrates the effectiveness of conditional GANs for structured image-to-image tasks. To further improve results, consider:

- Increasing training epochs
- Using better regularization or data augmentation
- Testing on other datasets like edges2shoes or maps

---

*Happy experimenting!*
